In [1]:
import sys
sys.path.append("..\src")

In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import time
from datetime import datetime

from sgcarmart import *

In [30]:
main_page_listing_list = []
for idx, link in enumerate(range(40)):
    url = "https://www.sgcarmart.com/used_cars/listing.php?BRSR=" + str(idx * 100) + "&RPG=100&AVL=2&VEH=2"
    main_page_listing_list.append(url)

In [31]:
print(main_page_listing_list,'\n','\n', len(main_page_listing_list))

['https://www.sgcarmart.com/used_cars/listing.php?BRSR=0&RPG=100&AVL=2&VEH=2', 'https://www.sgcarmart.com/used_cars/listing.php?BRSR=100&RPG=100&AVL=2&VEH=2', 'https://www.sgcarmart.com/used_cars/listing.php?BRSR=200&RPG=100&AVL=2&VEH=2', 'https://www.sgcarmart.com/used_cars/listing.php?BRSR=300&RPG=100&AVL=2&VEH=2', 'https://www.sgcarmart.com/used_cars/listing.php?BRSR=400&RPG=100&AVL=2&VEH=2', 'https://www.sgcarmart.com/used_cars/listing.php?BRSR=500&RPG=100&AVL=2&VEH=2', 'https://www.sgcarmart.com/used_cars/listing.php?BRSR=600&RPG=100&AVL=2&VEH=2', 'https://www.sgcarmart.com/used_cars/listing.php?BRSR=700&RPG=100&AVL=2&VEH=2', 'https://www.sgcarmart.com/used_cars/listing.php?BRSR=800&RPG=100&AVL=2&VEH=2', 'https://www.sgcarmart.com/used_cars/listing.php?BRSR=900&RPG=100&AVL=2&VEH=2', 'https://www.sgcarmart.com/used_cars/listing.php?BRSR=1000&RPG=100&AVL=2&VEH=2', 'https://www.sgcarmart.com/used_cars/listing.php?BRSR=1100&RPG=100&AVL=2&VEH=2', 'https://www.sgcarmart.com/used_cars/li

In [32]:
# Base url, or you can think of this as the individual car listing prefix
base_url = 'https://www.sgcarmart.com/used_cars/'
listing_urls = []

# Acquiring indvidual car listings    
for main_link in main_page_listing_list:
   
    # Make a request to the website and get the object
    content = requests.get(main_link)

    # Parse the HTML text
    soup = BeautifulSoup(content.text, 'lxml')

    # Find every single URL in the webpage , refer to this post: # https://stackoverflow.com/questions/46490626/getting-all-links-from-a-page-beautiful-soup
    # This returns a list of every tag that contains a link in one main link (each element in main page listing)
    links = soup.find_all('a')
    
    # Create a list for storing all the individual listing urls
    
    for link in links:
        # Get link in <a href>
        suffix = link.get('href')

        # Check if 'ID=' and 'DL=' exist in the string
        if ('ID=' in suffix) and ('DL=' in suffix):

            # Concatenate the two strings if they do
            listing_url = base_url + suffix
            
            # Append result to the list
            listing_urls.append(listing_url)
            
#     Removing duplicates
    set_listing_urls = set(listing_urls)
    listing_urls = list(set_listing_urls)
    
    # Prevent oneself from getting blocked from the website
    time.sleep(1)

In [33]:
print(len(listing_urls))
print(len(set(listing_urls)))
print(len(list(set(listing_urls))))

3985
3985
3985


In [34]:
print(listing_urls[:10])

['https://www.sgcarmart.com/used_cars/info.php?ID=1277757&DL=2025', 'https://www.sgcarmart.com/used_cars/info.php?ID=1211304&DL=3283&utm_content=SLeligible', 'https://www.sgcarmart.com/used_cars/info.php?ID=1269122&DL=3004&utm_content=SLeligible', 'https://www.sgcarmart.com/used_cars/info.php?ID=1096998&DL=3989&utm_content=SLeligible', 'https://www.sgcarmart.com/used_cars/info.php?ID=1259578&DL=3648&utm_content=SLeligible', 'https://www.sgcarmart.com/used_cars/info.php?ID=1230722&DL=3647&utm_content=SLeligible', 'https://www.sgcarmart.com/used_cars/info.php?ID=1118202&DL=2356&utm_content=SLeligible', 'https://www.sgcarmart.com/used_cars/info.php?ID=1277169&DL=1400', 'https://www.sgcarmart.com/used_cars/info.php?ID=1277993&DL=2984', 'https://www.sgcarmart.com/used_cars/info.php?ID=1263710&DL=1220&utm_content=SLeligible']


In [35]:
df = pd.DataFrame(columns=['LISTING_URL', 'BRAND', 'PRICE', 'DEPRE_VALUE_PER_YEAR',
       'REG_DATE', 'MILEAGE_KM', 'MANUFACTURED_YEAR',
       'ROAD_TAX_PER_YEAR','TRANSMISSION', 'DEREG_VALUE_FROM_SCRAPE_DATE',
       'SCRAPE_DATE', 'OMV', 'ARF', 'COE_FROM_SCRAPE_DATE',
       'DAYS_OF_COE_LEFT', 'ENGINE_CAPACITY_CC', 'CURB_WEIGHT_KG',
       'NO_OF_OWNERS', 'VEHICLE_TYPE'])

In [36]:
filename = 'sgcarmart_used_cars_prices3'
i = 0 # Indexing rows in the DF

for listingurl in listing_urls:
    response = requests.get(listingurl) 
    listing_url = BeautifulSoup(response.text, 'lxml')
    
    # Retrieval functions to pull data from the Individual Listings after they have been parsed
    df.loc[i, 'LISTING_URL'] = listingurl
    df.loc[i, 'BRAND'] = brand_retrieval(listing_url)
    df.loc[i, 'PRICE'] = price_retrieval(listing_url)

    try:
        df.loc[i, 'DEPRE_VALUE_PER_YEAR'] = depreciation_value_per_year_retrieval(listing_url)
    except:
        df.loc[i, 'DEPRE_VALUE_PER_YEAR'] = np.nan

    try:
        df.loc[i, 'REG_DATE'] = registered_date_retrieval(listing_url)
    except:
        df.loc[i, 'REG_DATE'] = np.nan

    try:
        df.loc[i, 'MILEAGE_KM'] = mileage_retrieval(listing_url)
    except:
        df.loc[i, 'MILEAGE_KM'] = np.nan

    try:
        df.loc[i, 'MANUFACTURED_YEAR'] = manufactured_year_retrieval(listing_url)
    except: 
        df.loc[i, 'MANUFACTURED_YEAR'] = np.nan
    
    try:
        df.loc[i, 'ROAD_TAX_PER_YEAR'] = road_tax_retrieval(listing_url)
    except:
        df.loc[i, 'ROAD_TAX_PER_YEAR'] = np.nan
    
    try:
        df.loc[i, 'TRANSMISSION'] = transmission_retrieval(listing_url)
    except:
        df.loc[i, 'TRANSMISSION'] = np.nan

    try:
        df.loc[i, 'DEREG_VALUE_FROM_SCRAPE_DATE'] = dereg_value_retrieval(listing_url)
    except: 
        df.loc[i, 'DEREG_VALUE_FROM_SCRAPE_DATE'] = np.nan
        
    df.loc[i, 'SCRAPE_DATE'] = datetime.now().strftime("%d/%m/%Y")
    
    try:
        df.loc[i, 'OMV'] = omv_retrieval(listing_url)
    except: 
        df.loc[i, 'OMV'] = np.nan

    try:
        df.loc[i, 'ARF'] = arf_retrieval(listing_url)
    except: 
        df.loc[i, 'ARF'] = np.nan
        
    try:
        df.loc[i, 'COE_FROM_SCRAPE_DATE'] = coe_retrieval(listing_url)
    except:
        df.loc[i, 'COE_FROM_SCRAPE_DATE'] = np.nan
        
    try:
        df.loc[i, 'DAYS_OF_COE_LEFT'] = days_of_coe_retrieval(listing_url)
    except:
        df.loc[i, 'DAYS_OF_COE_LEFT'] = np.nan
        
    try:
        df.loc[i, 'ENGINE_CAPACITY_CC'] = engine_capacity_retrieval(listing_url)
    except: 
        df.loc[i, 'ENGINE_CAPACITY_CC'] = np.nan

    try:
        df.loc[i, 'CURB_WEIGHT_KG'] = curb_weight_retrieval(listing_url)
    except:
        df.loc[i, 'CURB_WEIGHT_KG'] = np.nan
        
    try:
        df.loc[i, 'NO_OF_OWNERS'] = number_of_owners_retrieval(listing_url)
    except:
        df.loc[i, 'NO_OF_OWNERS'] = np.nan
        
    try:
        df.loc[i, 'VEHICLE_TYPE'] = type_of_vehicle_retrieval(listing_url)
    except:
        df.loc[i, 'VEHICLE_TYPE'] = np.nan
    
    df.to_csv("{}.csv".format(filename)) 
    i+=1
    time.sleep(1)




In [37]:
df = pd.read_csv('sgcarmart_used_cars_prices3.csv',index_col=0)
df.dropna()

,LISTING_URL,BRAND,PRICE,DEPRE_VALUE_PER_YEAR,REG_DATE,MILEAGE_KM,MANUFACTURED_YEAR,ROAD_TAX_PER_YEAR,TRANSMISSION,DEREG_VALUE_FROM_SCRAPE_DATE,SCRAPE_DATE,OMV,ARF,COE_FROM_SCRAPE_DATE,DAYS_OF_COE_LEFT,ENGINE_CAPACITY_CC,CURB_WEIGHT_KG,NO_OF_OWNERS,VEHICLE_TYPE
0,https://www.sgcarmart.com/used_cars/info.php?I...,Mitsubishi,92888.0,16350.0,08-Jan-2019,75333.0,2018,684.0,Auto,34590.0,13/03/2024,25699.0,27979.0,31101.0,1735.0,1499.0,1490.0,1.0,SUV
1,https://www.sgcarmart.com/used_cars/info.php?I...,Volvo,45800.0,16300.0,14-Mar-2016,77000.0,2016,684.0,Auto,25170.0,13/03/2024,24571.0,26400.0,46651.0,730.0,1498.0,1589.0,2.0,Luxury Sedan
2,https://www.sgcarmart.com/used_cars/info.php?I...,Honda,64800.0,13090.0,23-Feb-2009,161163.0,2008,1110.0,Auto,12737.0,13/03/2024,20211.0,20211.0,25727.0,1499.0,1595.0,1220.0,1.0,Mid-Sized Sedan
4,https://www.sgcarmart.com/used_cars/info.php?I...,Infiniti,87777.0,15290.0,23-Oct-2018,109000.0,2018,1202.0,Auto,38569.0,13/03/2024,30333.0,34467.0,31302.0,1679.0,1991.0,1644.0,2.0,Luxury Sedan
5,https://www.sgcarmart.com/used_cars/info.php?I...,Volvo,44600.0,14730.0,27-Apr-2016,105500.0,2016,684.0,Auto,26068.0,13/03/2024,24806.0,26729.0,47300.0,763.0,1498.0,1589.0,1.0,Luxury Sedan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3979,https://www.sgcarmart.com/used_cars/info.php?I...,Mazda,71800.0,15340.0,09-Mar-2018,135680.0,2017,1210.0,Auto,29336.0,13/03/2024,20855.0,21197.0,39001.0,1129.0,1998.0,1472.0,1.0,Luxury Sedan
3980,https://www.sgcarmart.com/used_cars/info.php?I...,Honda,51800.0,15770.0,10-Nov-2016,96000.0,2016,742.0,Auto,25781.0,13/03/2024,19689.0,19689.0,52503.0,947.0,1597.0,1249.0,2.0,Mid-Sized Sedan
3981,https://www.sgcarmart.com/used_cars/info.php?I...,Lexus,134388.0,21670.0,10-May-2019,29900.0,2019,1198.0,Auto,58670.0,13/03/2024,38007.0,45210.0,48000.0,1861.0,1987.0,1485.0,2.0,SUV
3982,https://www.sgcarmart.com/used_cars/info.php?I...,BMW,188800.0,37820.0,21-Jun-2017,129000.0,2017,2384.0,Auto,102429.0,13/03/2024,87886.0,130195.0,54616.0,1192.0,2998.0,1770.0,3.0,Luxury Sedan
